In [1]:
# Reload modules automatically on changes
%load_ext autoreload
%autoreload 2

In [2]:
import tsdf
import os
import sys
import numpy as np

In [3]:
DATA_DIR = "../tests/data"
metadata = tsdf.load_from_path(os.path.join(DATA_DIR, "dummy_10_3_int16.json"))
data_in = metadata["dummy_10_3_int16.bin"].load_binary()
print(data_in.dtype, data_in.shape)

int16 (10, 3)


In [4]:
# Perform some advanced data processing
data_out = (data_in / 10).astype('float32')
print(data_out.dtype, data_out.shape)

float32 (10, 3)


In [5]:
bin_meta = tsdf.get_metadata_from_ndarray(data_out)

{'data_type': 'float', 'bits': 32, 'endianness': 'little'}


In [6]:
tsdf.save_binary_file('dummy_output.bin', data_out)

meta = extract_fields(original_metadata, ['study_id', 'sefsef'])

new_metadata = MetaData([meta, bin_meta])

new_metadata = MetaData({
    study_id: original_metadata['study_id'],
    ...
    bits: bin_meta['bits']
    ...
})